<a href="https://colab.research.google.com/github/Dieljr/pythonDev/blob/master/HousePriceIowa_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nesta série vou investigar o dataset House Price of Aime, Iwoa.  Ao contrário do PIMA INDIANS DIABETES (último post) este conjunto é mais complexo, principalmente do ponto de vista da engenharia de dados.

Fonte: http://jse.amstat.org/v19n3/decock.pdf da pesquisa.
Fonte: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data do dataset

In [ ]:
# Carregar pacotes iniciais

# explorar dados
import pandas as pd

# estatísticas
import numpy as np

# Visualizações
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 400

import seaborn as sb

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
print(type(train))
print(type(test))

In [ ]:
print(train.shape)
print(test.shape)

São incríveis 80 colunas de variáveis independentes e 1 dependente (SalePrice)!
Então o objetivo de negócio é prever o preço o imóvel baseados nas melhores variáveis.

In [ ]:
train.info()

In [ ]:
del train['Id']

In [ ]:
del test['Id']

In [ ]:
# valores NAN plotados
mpl.rcParams['figure.figsize'] = (15,10)
plt.hist(train.isna().sum())

In [ ]:
# Função que verifica como estão divididas as variáveis qualitativas por subcategorias?
def unicos_obj(df):
    for i in df.select_dtypes(include = 'object'):
        print(i, '', df[i].unique())
unicos_obj(train)
unicos_obj(test)

In [ ]:
# Função que converte 'other' e suas VARIAÇÕES em 'Other'
def na_other_obj(df):
    for i in df:
        df[i].replace('Oth', 'Other', inplace = True)
        df[i].replace('OthW', 'Other', inplace = True)
        df[i].replace('Othr', 'Other', inplace = True)
        df[i].replace('other', 'Other', inplace = True)
        
na_other_obj(train)
na_other_obj(test)

In [ ]:
# Função que identifica variáveis com alto coeficiente de variação (> 25%)
med_x = []
std_x = []
cv_x = []
var_i = []
def analise_cv(df):
    cont_std = 0
    for i in df.select_dtypes(include = 'number'):
        
        med, std = df[i].mean(), df[i].std() # média e desvio pádrão
        cv = (std / med)*100 # coeficiente de variação
        
        var_i.append(i)
        med_x.append(med)
        std_x.append(std)
        cv_x.append(cv)
        
        if cv > 25: # valor de corte
            cont_std += 1
            
            #print(i)
            #print('média de ',round(med))
            #print('desvio padrão de ', round(std))
            #print('Coeficiente de variação = ',  round(cv))
            #print('')   
            
    
    print('O conjunto de dados tem ', cont_std, ' variáveis com desvio padrão acima de 25%')
    #print('')
    
    
analise_cv(train)

In [ ]:
# Tabela com média, desvio padrão e coeficiente de variação de cada variável numérica
print(pd.DataFrame({'variavel':var_i,
                       'media':med_x,
                       'desvio_padrao': std_x,
                       'coeficiente_variacao': cv_x}))

In [ ]:
# Função que verifica valores missing das variáveis numéricas
def missing_num(df):
    for i in df:
        print(i, '', df[i].isna().sum())

missing_num(train.select_dtypes(include = 'number'))
missing_num(test.select_dtypes(include = 'number'))

In [ ]:
# Função que substitui os valores MISSING pela média de cada coluna numérica
def na_media(df):
    for i in df:
        df[i].fillna(df[i].mean(), inplace = True)
        

na_media(train.select_dtypes(include = 'number'))
na_media(test.select_dtypes(include = 'number'))

In [ ]:
# identificação de OUTLIERS em uma distribuição normal

from IPython.display import Image
Image ('outlier normal distribution.png')

In [ ]:
# Função para plotagem de valores outliers
def box_plot(df):
    plt.figure(figsize = (10,8))
    df.boxplot()
    plt.grid(True)
    plt.show()

In [ ]:
# plot valores numéricos outliers no data set treino
box_plot(train.select_dtypes(include = 'number'))

In [ ]:
# plot valores numéricos outliers no data set test
box_plot(test.select_dtypes(include = 'number'))

In [ ]:
preco_antigo = train['SalePrice'] # para usar na comparação ante e depois dos autliers e da normalização (log10)

In [ ]:
# pacote para remoção de outliers
from scipy.stats import zscore, iqr

In [ ]:
# Função que elimina valores extremos (0,25 < x > 0.75).
def remove_outlier(df):
    
    iqr(df, axis = 0)

In [ ]:
# chamada à função de remoção de outliers
# inclui um filtro para variáveis númericas
        
remove_outlier(train.select_dtypes(include = 'number'))
remove_outlier(test.select_dtypes(include = 'number'))

In [ ]:
# função que normaliza o dataset

def normal(df):
    for i in df.select_dtypes(include = 'number'):
        df[i] = np.log10(df[i])

normal(train)
normal(test)

In [ ]:
# compara o antes e o depois da normalização

comp_precos = {'original':preco_antigo, 'normalizado':train['SalePrice']}
df_plot = pd.DataFrame(comp_precos)
mpl.rcParams['figure.figsize'] = (20,8)
df_plot.hist()

In [ ]:
# Mostra as colunas categóricas para serem usadas na trasnformação (np.get_dummies)
print(train.select_dtypes(include = 'object').columns)
print('')
print(test.select_dtypes(include = 'object').columns)

In [ ]:
#Aplicar transformação das variáveis categóricas para numéricas

train = pd.get_dummies(train, columns = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'])

test = pd.get_dummies(test, columns = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'])

#                                    ANÁLISE APÓS TRATAMENTOS

In [ ]:
train.describe()

In [ ]:
# Remover os valores infinitos (inf, -inf)
train = train.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)

In [ ]:
train.describe()

In [ ]:
train.fillna(train.mean(), inplace = True)

In [ ]:
test.fillna(test.mean(), inplace = True)

In [ ]:
test

# ENTRAREMOS NO PREPROCESSAMENTO

In [ ]:
# subconjunto com variáveis independentes
X = train.drop(['SalePrice'], axis = 1)

# subconjunto com a variável dependente
y = train['SalePrice']

In [ ]:
train

In [ ]:
# Veficar as dimensões dos subconjuntos

print(X.shape, y.shape)

In [ ]:
# Carregar pacotes para preprocessamento e machine learning

from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LassoCV, Ridge

from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR

# explained_variance_score: essa metrica é a função da pontuação da regressão da variancia explicada: varia de 0 a 1, 
# sendo 1 = ótimo.
# na minha opinião, o mais importante para esse dataset é manter o padrão (uniform_average), mas você pode tentar outros

# A outra métrica é r2_score: todos nós já sabemos como é importante o valor de R2, que varia de 0 a 1.
from sklearn.metrics import explained_variance_score, r2_score, SCORERS

from scipy.stats.stats import pearsonr

import warnings
warnings.filterwarnings("ignore")


# MACHINE LEARNING: Linear Regression

In [ ]:
# Função para testar tamanho da amostra e n_component do PCA no algoritmo de Regressão Linear

rs = []
nc = []
r2s = []
scoreev = []

for i in range(10, 30):
    for j in range(100, 500, 10):
    # separar treinos e testes
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = j)
        soma_j = 100 + j

        lr = LinearRegression()
        evs = explained_variance_score

        # Aplicar automação para n_component
        pca = PCA(n_components=i)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)
        taxas = pca.explained_variance_ratio_

        # Treinar o modelo
        model_lr = lr.fit(X_train, y_train)

        # Criar o modelo de predição
        pred_lr = lr.predict(X_test)
        pred_lr.shape

        # Avaliar o modelo com r2_score
        r2 = r2_score(y_true = y_test, y_pred = pred_lr)

        # Avaliar o modelo com explained_variance_score
        score_ev = evs(y_true = y_test, y_pred = pred_lr)
        
        rs.append(soma_j)
        nc.append(i)
        r2s.append(r2)
        scoreev.append(score_ev)

In [ ]:
scores_lr = pd.DataFrame(columns = ['n_component','random_state', 'r2_score', 'evs'], data = (list(zip(nc, rs, r2s, scoreev))))

In [ ]:
# n-component e ramdom_state pulam de 10 em 10
scores_lr.head(10)

In [ ]:
# apresentação do melhor escore entre a variação de n_component e random_state
melhor_score_lr_ncomp = scores_lr.groupby('n_component')[['r2_score', 'evs']].max(); melhor_score_lr_ncomp.max()

In [ ]:
# Resumo dos escores
scores_lr.max()

In [ ]:
# Função que avalia o rmse (root mean squared error) com Cross Validation
def rmse(modelo):
    rmse = np.sqrt(-cross_val_score(modelo,
                               X_train,
                               y_train,
                               scoring = 'neg_mean_squared_error',
                               cv = 5))
    return(rmse)

In [ ]:
# Resultado das cinco rodadas do erro quadrado médio
rmse(model_lr)

In [ ]:
# Resultado da média do RMSE
rmse(model_lr).mean()

In [ ]:
# Visualização da reta de regressão treinada
sb.jointplot(x = y_test, y = pred_lr, kind = 'reg')

In [ ]:
# Visualização do resumo estatísitco
import statsmodels.api as sm
import statsmodels.formula as smf
from statsmodels.regression.linear_model import OLSResults

reg = sm.OLS(y, X).fit()

print(reg.summary())

In [ ]:
# Cria uma série de dados
reg_coef = pd.Series(reg.pvalues, index = X.columns)

In [ ]:
# Concatena duas lista de dados para selecionar os menores valores de P-Values
coef_imp = pd.concat([reg_coef.sort_values(ascending = False).head(0), reg_coef.sort_values(ascending = True).head(34)])

In [ ]:
# Cria um plot das variáveis com valor de p (< 0.05)
# são 34 variáveis com p_value desejável
plt.figure(figsize=(10,8))
coef_imp.sort_values().plot(kind = 'bar')

plt.ylabel('Análise p < 0.05')
plt.title('Melhores Variáveis para Linear Regression com P-Value < 0.05')
plt.plot([0.05,0.05,0.05,0.05,0.05,0.05], 'r--')
plt.show()


In [ ]:
# Concatena duas lista de dados para selecionar os maiores valores de P-Values
coef_imp = pd.concat([reg_coef.sort_values(ascending = False).head(254), reg_coef.sort_values(ascending = True).head(0)])

In [ ]:
# Cria um plot das variáveis com valor de p (< 0.05)
# são 34 variáveis com p_value desejável
plt.figure(figsize=(10,8))
coef_imp.sort_values().plot(kind = 'bar')

plt.ylabel('Análise p < 0.05')
plt.title('Piores Variáveis para Linear Regression com P-Value > 0.05')
plt.plot([0.05,0.05,0.05,0.05,0.05,0.05], 'r--')
plt.show()


# MACHINE LEARNING: Lasso

Lasso significa "Least Absolute Shrinkage and Selection Operator" ou operação(regularização e regressão) e seleção mínimo absoluto.
Isso siginifica que LASSO faz a regularização das variáveis, bem como a seleção destas em ordem de acurácia.

In [ ]:
# subconjunto com variáveis independentes
X = train.drop(['SalePrice'], axis = 1)

# subconjunto com a variável dependente
y = train['SalePrice']

In [ ]:
# Função para testar melhor amostragem para LASSO

rs_ls = []
r2s_ls = []
scoreev_ls = []


for j in range(100, 500, 10):
    # separar treinos e testes
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = j)
    soma_j = 100 + j
    
    # Manter os parametros padrão
    ls = LassoCV(cv = 5, random_state = 100, alphas = [1, 0.01, 0.001, 0.0005])
    
    # Treinar o modelo
    model_ls = ls.fit(X_train, y_train)
    model_ls.score(X, y)

    # Criar o modelo de predição
    pred_ls = model_ls.predict(X_test)

    # Avaliar o modelo com r2_score
    r2_ls = r2_score(y_true = y_test, y_pred = pred_ls)

    # Avaliar o modelo com explained_variance_score
    score_ls = evs(y_true = y_test, y_pred = pred_ls)
    
    rs_ls.append(soma_j)
    r2s_ls.append(r2_ls)
    scoreev_ls.append(score_ls)

In [ ]:

scores_ls = pd.DataFrame(columns = ['random_state', 'r2_score', 'evs'], data = (list(zip(rs_ls, r2s_ls, scoreev_ls))))

In [ ]:
scores_ls.head()

In [ ]:
melhor_score_ls_ncomp = scores_ls.groupby('random_state')[['r2_score', 'evs']].max(); melhor_score_ls_ncomp.max()

In [ ]:
# Resumo dos escores
scores_ls.max()

In [ ]:
# Erro médio qudrado para o modelo LASSO
rmse(model_ls)

In [ ]:
# Erro médio do modelo LASSO
print(rmse(model_ls).mean())

In [ ]:
lasso_coef = pd.Series(model_ls.coef_, index = X_train.columns)

In [ ]:
ls_melhor_coef = pd.concat([lasso_coef.sort_values(ascending = True).head(5), lasso_coef.sort_values(ascending = False).head(20)])

In [ ]:
plt.figure(figsize=(12,8))
ls_melhor_coef.plot(kind = 'barh')
plt.xlabel('Valor')
plt.title('Representatitvidade das Melhores Variáveis')

In [ ]:
ax = plt.gca()

ax.plot(alphas*2, coef)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')


In [ ]:
ax = plt.gca()

ax.plot(alphas, lasso_coef)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('Standardized Coefficients')
plt.title('Lasso coefficients as a function of alpha');

# MACHINE LEARNING: SVM

SVM ou Support Vector Machine: Esse algoritmo, após análise, reconhe os padrões. É um algoritmo não probabilístico. 
Ele faz o tratamento de outliers.Não é aconselhado para dataset muito grandes. Os suportes são os separadores dos grupos.    

In [ ]:
# subconjunto com variáveis independentes
X = train.drop(['SalePrice'], axis = 1)

# subconjunto com a variável dependente
y = train['SalePrice']

In [ ]:
# Função para testar tamanho da amostra e n_component do PCA no algoritmo de Regressão Linear

rs_svm = []
r2s_svm = []
scoreev_svm = []


for j in range(100, 500, 10):
    # separar treinos e testes
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = j)

    # Normalizar datasets
    # Só normalizamos a variável dependente
    #sc = StandardScaler()
    #X_train = sc.fit_transform(X_train)
    #X_test = sc.transform(X_test)

    svm = LinearSVR()

    # Treinar o modelo
    model_svm = svm.fit(X_train, y_train)

    # Criar o modelo de predição
    pred_svm = svm.predict(X_test)

    # Avaliar o modelo com r2_score
    r2_svm = r2_score(y_true = y_test, y_pred = pred_svm)

    # Avaliar o modelo com explained_variance_score
    score_svm = evs(y_true = y_test, y_pred = pred_svm)

    rs_svm.append(soma_j)
    r2s_svm.append(r2)
    scoreev_svm.append(score_svm)

In [ ]:
scores_svm = pd.DataFrame(columns = ['random_state', 'r2_score', 'evs'], data = (list(zip(rs_svm, r2s_svm, scoreev_svm))))

In [ ]:
scores_svm.head()

In [ ]:
melhor_score_svm_ncomp = scores_svm.groupby('random_state')[['r2_score', 'evs']].max(); melhor_score_ls_ncomp.max()

In [ ]:
# Resumo dos escores
scores_svm.max()

In [ ]:
svm_coef = pd.Series(model_svm.coef_, index = X.columns)

In [ ]:
svm_melhor_coef = pd.concat([svm_coef.sort_values(ascending = True).head(10), svm_coef.sort_values(ascending = False).head(10)])

In [ ]:
plt.figure(figsize=(12,8))
svm_melhor_coef.plot(kind = 'barh')
plt.xlabel('Valor')
plt.title('Melhores Variáveis')

In [ ]:
# Plotagem da comparação da regressão dos modelos testados
plt.figure(figsize = (10,8))

sb.jointplot(x = y_test, y = pred_lr, kind = 'reg', color='b')
plt.ylabel('Linear Regression')
plt.legend(['regressão', 'novos dados'], ncol = 2, loc = "lower right")

sb.jointplot(x = y_test, y = pred_ls, kind = 'reg', color = 'g')
plt.ylabel('Lasso - Cross Validation')
plt.legend(['regressão', 'novos dados'], ncol = 2, loc = "lower right")

sb.jointplot(x = y_test, y = pred_svm, kind = 'reg', color = 'r')
plt.ylabel('Support Vector Machine')
plt.legend(['regressão', 'novos dados'], ncol = 2, loc = "lower right")

plt.suptitle('Comparação entre os modelos Linear Regression, Lasso e SVM')
plt.show()

In [ ]:
plt.show